In [142]:
import pandas as pd
from sklearn import linear_model
import numpy as np
from scipy.stats import variation
from itertools import chain, combinations
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso

### Question 1

#### E

In [2]:
simple_reg = pd.read_csv('SimpleReg.csv')

In [4]:
simple_reg.head()

,x,y
0,0.997399,0.002905
1,0.621641,0.346669
2,0.715032,0.127638
3,0.463848,0.683570
4,0.650216,0.288742


In [10]:
X = simple_reg['x'].values.reshape(-1, 1)
y = simple_reg['y'].values.reshape(-1, 1)

In [11]:
regr = linear_model.LinearRegression(fit_intercept=False)
regr.fit(X, y)

LinearRegression(fit_intercept=False)

In [12]:
print('Coefficients: \n', regr.coef_)

Coefficients: 
 [[-0.09245499]]


In [18]:
coefficient = regr.coef_[0][0]

In [26]:
Sxx = sum(simple_reg['x'] * simple_reg['x'])

In [34]:
CV = sum((simple_reg['y'] - coefficient*(simple_reg['x']) / (1 - ((simple_reg['x']*simple_reg['x']) / Sxx)))**2) / len(simple_reg['x'])

In [36]:
print('CV = ' + str(CV))

CV = 0.7232993202119912


In [44]:
cv_scipy = variation(simple_reg, axis=0)

In [45]:
print('CV Scipy = ' + str(cv_scipy))

CV Scipy = [-31.85344957   1.01721531]


### Question 2

#### A

In [46]:
Train = pd.read_csv('HW4TrainData.csv')
Test = pd.read_csv('HW4TestData.csv')
Validation = pd.read_csv('HW4ValidationData.csv')

In [49]:
Train.columns

Index(['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'y'], dtype='object')

In [99]:
y_Train = Train['y']
x_Train = Train.drop(columns=['y'])

In [100]:
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

In [101]:
complete_variable_list = list(powerset(x_Train))

In [102]:
i = 0
best_variable_list = []
regr_q2 = linear_model.LinearRegression()
best_mse = 100000

for x in complete_variable_list:
    used_variable_list = []
    for variables in complete_variable_list[i]:
        used_variable_list.append(variables)
    regr_q2.fit(x_Train[used_variable_list],y_Train)
    q2_pred = regr_q2.predict(Validation[used_variable_list])
    q2_true = Validation['y']
    mse_value = mean_squared_error(q2_true,q2_pred)
    if(mse_value < best_mse):
        best_mse = mse_value
        best_variable_list = used_variable_list
    i = i + 1

In [103]:
print('best combination of variables: ' + str(best_variable_list))
print('best MSE: ' + str(best_mse))

best combination of variables: ['x6', 'x8', 'x9']
best MSE: 0.11224169059923915


#### E

In [104]:
y_Values = Test['y']

In [108]:
ModelA = Test[['x6','x8','x9']]
ModelB = Test[['x5','x7','x9']]
ModelC = Test[['x1','x5','x6','x7','x9','x10']]
ModelD = Test[['x5','x6','x7','x10']]

In [112]:
regr_q2e = linear_model.LinearRegression()

regr_q2e.fit(x_Train[['x6','x8','x9']],y_Train)
q2e_pred = regr_q2e.predict(ModelA)
print('MSE Model A: ' + str(mean_squared_error(y_Values,q2e_pred)))

regr_q2e.fit(x_Train[['x5','x7','x9']],y_Train)
q2e_pred = regr_q2e.predict(ModelB)
print('MSE Model B: ' + str(mean_squared_error(y_Values,q2e_pred)))

regr_q2e.fit(x_Train[['x1','x5','x6','x7','x9','x10']],y_Train)
q2e_pred = regr_q2e.predict(ModelC)
print('MSE Model C: ' + str(mean_squared_error(y_Values,q2e_pred)))

regr_q2e.fit(x_Train[['x5','x6','x7','x10']],y_Train)
q2e_pred = regr_q2e.predict(ModelD)
print('MSE Model D: ' + str(mean_squared_error(y_Values,q2e_pred)))


MSE Model A: 0.1407035309388078
MSE Model B: 0.14112115653565416
MSE Model C: 0.1441947011671233
MSE Model D: 0.14433807231950413


### Question 3

In [114]:
partial_auto = pd.read_csv('PartialAuto.csv')

In [115]:
partial_auto

,mpg,cylinders,displacement,horsepower,weight,acceleration
0,22.4,6,231.0,110,3415,15.8
1,29.0,4,135.0,84,2525,16.0
2,19.0,4,122.0,85,2310,18.5
3,44.3,4,90.0,48,2085,21.7
4,19.4,8,318.0,140,3735,13.2
...,...,...,...,...,...,...
387,21.5,4,121.0,110,2600,12.8
388,14.0,8,340.0,160,3609,8.0
389,29.8,4,134.0,90,2711,15.5
390,20.0,6,225.0,100,3651,17.7


In [117]:
auto_y = partial_auto['mpg']
auto_X = partial_auto.drop(columns=['mpg'])

In [118]:
auto_X_train, auto_X_test, auto_y_train, auto_y_test = train_test_split(auto_X, auto_y, test_size=92, random_state=1)

In [121]:
auto_regr = linear_model.LinearRegression()

auto_regr.fit(auto_X_train,auto_y_train)
auto_pred = auto_regr.predict(auto_X_test)
print('MSE : ' + str(mean_squared_error(auto_y_test,auto_pred)))

MSE : 18.436486007078884


In [124]:
n_iterations = 1000
lambda_list = np.logspace(-3, 5, n_iterations)
alphas =  lambda_list
ridge_cv = RidgeCV(alphas=alphas, cv=10)
ridge_cv.fit(auto_X_train, auto_y_train)
print(ridge_cv.alpha_)

24175.940791691282


In [140]:
Ridge_updated = Ridge(alpha=24175.940791691282)

In [141]:
Ridge_updated.fit(auto_X_train, auto_y_train)
auto_pred = Ridge_updated.predict(auto_X_test)
print('MSE : ' + str(mean_squared_error(auto_y_test,auto_pred)))

MSE : 18.14091149997406


In [143]:
lasso_cv = LassoCV(alphas=alphas, cv=10)
lasso_cv.fit(auto_X_train, auto_y_train)
print(lasso_cv.alpha_)

0.5791122647641759


In [144]:
Lasso_updated = Lasso(alpha=0.5791122647641759)

In [145]:
Lasso_updated.fit(auto_X_train, auto_y_train)
auto_pred = Lasso_updated.predict(auto_X_test)
print('MSE : ' + str(mean_squared_error(auto_y_test,auto_pred)))

MSE : 18.23546997235851
